In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install pymorphy2[fast]
!pip install navec

In [117]:
from transformers import BertTokenizer
from transformers import AdamW, BertForMaskedLM
import torch
import numpy as np
import pymorphy2
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from navec import Navec
from typing import List
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import KeyedVectors
import scipy.spatial.distance as cos_dist
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import f1_score
from keras import backend as K
import tensorflow as tf
from tqdm import tqdm
import json
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from IPython.display import clear_output
import matplotlib.pyplot as plt

random.seed(44)
morph = pymorphy2.MorphAnalyzer()
nltk.download('punkt')
navec = Navec.load(save_path + '/navec_hudlit_v1_12B_500K_300d_100q.tar')
save_path = '/content/drive/MyDrive'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using cpu


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForMaskedLM.from_pretrained("bert-base-multilingual-cased")

In [ ]:
categories = ("животные", "музыка", "спорт", "литература")
categories_eng = ("animals", "music", "sport", "literature")

nouns_actors = []
for cat in categories_eng:
  with open(save_path + f'/keywords/actors/true_keywords_nouns_actors_{cat}.txt', 'r') as f:
    words = f.read()
  nouns_actors.append(words.split('\n')[:-1])

nouns = []
for ind, cat in enumerate(categories_eng):
  with open(save_path + f'/keywords/nouns/true_keywords_nouns_{cat}.txt', 'r') as f:
    words = f.read()
  words = words.split('\n')[:-1]
  nouns.append(words)

In [101]:
class MaskCreator():

  def __init__(self):
    self.tokenizer = nltk.WordPunctTokenizer()
    self.bigram_mod = gensim.models.Phrases.load(save_path + '/bigram_model.pkl')
    self.keywords = set()
    self.cat_keywords = [set() for i in range(4)]
    for ind, cat in enumerate(categories_eng):
      with open(save_path + f'/keywords/nouns/true_keywords_nouns_{cat}.txt', 'r') as file:
        self.keywords |= set([line.strip() for line in file.readlines()])
        self.cat_keywords[ind] |= set([line.strip() for line in file.readlines()])

  def make_bigrams(self, doc):
      return self.bigram_mod[doc]

  def mask(self, text, category=4) -> (str, List[str]):
    masks_dict = []
    tokens = self.tokenizer.tokenize(text.lower())
    tokens_normal = [morph.parse(w)[0].normal_form for w in tokens]
    tokens_bigrammed = self.make_bigrams(tokens_normal)
    
    if len(tokens_bigrammed) < len(tokens):
      ind_go = 0
      for i in range(len(tokens_bigrammed)):
        if tokens_normal[ind_go] != tokens_bigrammed[i]:
          tokens = tokens[:ind_go] + [tokens_bigrammed[i]] + tokens[ind_go+2:]
          ind_go += 2
        else:
          ind_go += 1

    if category == 4:
      now_keywords = self.keywords
    else:
      now_keywords = self.cat_keywords[category]

    for ind, token in enumerate(tokens):
      word = morph.parse(token.lower())[0].normal_form
      if word in now_keywords:
        if word not in masks_dict:
          masks_dict.append(word)
        tokens[ind] = 'mask' + str(masks_dict.index(word, 0))
    text = nltk.tokenize.treebank.TreebankWordDetokenizer().detokenize(tokens)
    text = text.replace(' .', '.')
    return text, masks_dict

In [102]:
MaskCreator().mask('Книги. книга. Футбольное поле, футбольных полей, бассейн бассейне, олимпийская игра.')

('mask0. mask0. футбольное mask1, футбольных mask1, mask2 mask2, олимпийский_игра.',
 ['книга', 'поле', 'бассейн'])

In [ ]:
df = pd.read_csv(save_path + '/facts2.csv')
df_new = df
df = pd.read_csv(save_path + '/dataset_disclosed.csv', sep=';')
df = df.drop('id', 1)
df = df.rename(columns={'task': 'texts', 'category': 'labels'})
df.labels = df.labels.replace('животные', 0).replace('музыка', 1).replace('спорт', 2).replace('литература', 3)
df_new2 = df
df = pd.read_csv(save_path + '/full_marked_dataset.csv')
df = df.drop('Unnamed: 0', 1)
df = df.rename(columns={'text': 'texts', 'ans': 'labels'})
df.labels = df.labels.replace('животные', 0).replace('музыка', 1).replace('спорт', 2).replace('литература', 3).replace('неизвестно', 4)
df = df.drop([i for i in range(len(df.labels)) if df.labels[i] == 4], 0)
df = df.reset_index(drop=True)
for i in range(len(df_new)):
  df.loc[-i-1] = df_new.loc[i]
df = df.reset_index(drop=True)
for i in range(len(df_new2)):
  df.loc[-i-1] = df_new.loc[i]
df = df.reset_index(drop=True)
df

,texts,labels
0,\t1.1. Летом в спортивный лагерь ходили 50 дет...,2
1,\t1.2. На концерте в летнем лагере ребята игра...,1
2,\t1.4. Во время летних соревнований по плавани...,2
3,\t1.5. В спортивную школу во время летних кани...,2
4,\t1.7. Самые крупные животные на нашей планете...,0
...,...,...
2417,"2.Спорт не стоит на месте, а движется вперед в...",2
2418,3. Активные спортивные занятия положительно вл...,2
2419,"4. Спорт — лучшая профилактика депрессии, диаб...",2
2420,5. Для пожилых людей самыми полезными видами с...,2


In [103]:
with open(save_path + '/aiijc_comand_data.json', 'r') as f :
  data = json.load(f)
df_wiki = pd.DataFrame(data)

df_wiki

,texts,links
животные,[Живо́тные (лат. Animalia) — традиционно (со в...,"[Волосатики, Национальная парламентская библио..."
музыка,"[Му́зыка (греч. μουσική, субстантивированное п...","[Мини-альбом, Волчья квинта, Ре (нота), Музыка..."
спорт,"[Спорт (англ. sport, сокращение от первоначаль...","[Олимпийская хартия, Болельщик, Ольмеки, Шаоли..."
литература,"[Литерату́ра (лат. lit(t)eratura, — написанное...","[Детская литература, Эфиопская литература, Сов..."


In [104]:
X = []
Y = []

alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')
for ind, text in tqdm(enumerate(df_wiki['texts'])):
  for sentence in text:
    sentence = sentence.lower().replace('\n', ' ')
    sentence = ''.join(c for c in sentence if c in alphabet)
    sentence = sentence.split()
    for word in sentence:
      #word = morph.parse(word)[0].normal_form
      if not word in navec:
        continue
      X.append(navec[word])
      #Y.append([0 if i != ind else 1 for i in range(4)])
      Y.append(ind)

X = np.array(X)
Y = np.array(Y)

4it [00:36,  9.14s/it]


In [ ]:
X = []
Y = []

alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')
for ind, text in tqdm(enumerate(df['texts'])):
  label = df['labels'][ind]
  text = text.lower().replace('\n', ' ')
  text = ''.join(c for c in text if c in alphabet)
  text = text.split()
  for word in text:
    #word = morph.parse(word)[0].normal_form
    if not word in navec:
      continue
    X.append(navec[word])
    Y.append([0 if i != label else 1 for i in range(4)])

X = np.array(X)
Y = np.array(Y)

4it [00:00, 12.96it/s]


In [105]:
X.shape, Y.shape

((3026268, 300), (3026268,))

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

word_model = Sequential()
word_model.add(Dense(100, input_dim=300, activation='relu'))
word_model.add(Dense(4, activation = 'softmax'))
word_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1_m])

In [ ]:
word_model.fit(X, Y, epochs=10, batch_size=256)

Epoch 1/10
11822/11822 [==============================] - 41s 3ms/step - loss: 1.2608 - accuracy: 0.3978 - f1_m: 0.1899
Epoch 2/10
11822/11822 [==============================] - 40s 3ms/step - loss: 1.2569 - accuracy: 0.4004 - f1_m: 0.1965
Epoch 3/10
11822/11822 [==============================] - 40s 3ms/step - loss: 1.2546 - accuracy: 0.4018 - f1_m: 0.1993
Epoch 4/10
11822/11822 [==============================] - 38s 3ms/step - loss: 1.2530 - accuracy: 0.4028 - f1_m: 0.2020
Epoch 5/10
11822/11822 [==============================] - 39s 3ms/step - loss: 1.2518 - accuracy: 0.4036 - f1_m: 0.2037
Epoch 6/10
11822/11822 [==============================] - 39s 3ms/step - loss: 1.2509 - accuracy: 0.4041 - f1_m: 0.2055
Epoch 7/10
11822/11822 [==============================] - 39s 3ms/step - loss: 1.2502 - accuracy: 0.4048 - f1_m: 0.2067
Epoch 8/10
11822/11822 [==============================] - 39s 3ms/step - loss: 1.2496 - accuracy: 0.4048 - f1_m: 0.2077
Epoch 9/10
11822/11822 [================

In [ ]:
word_model.predict(np.array([navec['турпоход']]))

array([[0.30372334, 0.14590675, 0.38346455, 0.16690543]], dtype=float32)

In [108]:
class NN(nn.Module):
  def __init__(self):
    super(NN, self).__init__()
    self.l1 = nn.Linear(300, 100)
    self.l2 = nn.ReLU()
    self.l3 = nn.Linear(100, 4)

  def forward(self, x):
    x = self.l1(x)
    x = self.l2(x)
    x = self.l3(x)
    return x

In [111]:
X = torch.tensor(X)
Y = torch.tensor(Y)
train_data = TensorDataset(X, Y)
train_dataloader = DataLoader(
  train_data,
  sampler=RandomSampler(train_data),
  batch_size=128
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [113]:
word_model = NN()
word_model = word_model.to(device)
print(word_model)

NN(
  (l1): Linear(in_features=300, out_features=100, bias=True)
  (l2): ReLU()
  (l3): Linear(in_features=100, out_features=4, bias=True)
)


In [115]:
criterion = nn.CrossEntropyLoss() 
criterion = criterion.to(device)
optimizer = optim.Adam(word_model.parameters(), lr=1e-3)

In [119]:
from IPython.display import clear_output

train_loss_set = []

word_model.train()

for step, batch in enumerate(tqdm(train_dataloader)):
  batch = tuple(t.to(device) for t in batch)
  b_inputs, b_labels = batch
  
  optimizer.zero_grad()
  
  b_pred = word_model(b_inputs)

  loss = criterion(b_pred, b_labels)

  train_loss_set.append(loss.item())

  loss.backward()
  
  optimizer.step()
  
  '''clear_output(True)
  plt.plot(train_loss_set)
  plt.title("Training loss")
  plt.xlabel("Batch")
  plt.ylabel("Loss")
  plt.show()'''

100%|██████████| 23643/23643 [00:51<00:00, 459.28it/s]


In [139]:
word_model.eval()
now = word_model(torch.tensor([navec['поход']]))
now

tensor([[-1.3445, -0.1988,  0.6808,  0.1844]], grad_fn=<AddmmBackward>)

In [171]:
now_word = 'сколько'
a = cos_dist.cosine(navec[now_word], navec['животные'])
b = cos_dist.cosine(navec[now_word], navec['музыка'])
c = cos_dist.cosine(navec[now_word], navec['спорт'])
d = cos_dist.cosine(navec[now_word], navec['литература'])
s = nn.Softmax()
print(s(torch.tensor([1-a, 1-b, 1-c, 1-d])), 1-a)
now = word_model(torch.tensor([navec[now_word]]))
s(now)

tensor([0.2512, 0.2569, 0.2375, 0.2544], dtype=torch.float64) 0.08583778887987137


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


tensor([[0.1766, 0.2362, 0.2802, 0.3070]], grad_fn=<SoftmaxBackward>)

In [129]:
#sentence = '4.30. У слона пульс 20 ударов в минуту, а у паука на 40 ударов в минуту больше. Какой пульс у паука?'
#sentence = '   1.1. Летом в спортивный лагерь ходили 50 детей, из них 9 девочек. Сколько мальчиков ходили в спортивный лагерь?'
sentence = '   1.4. Во время летних соревнований по плаванию ребята посетили бассейн. Длина плавательной дорожки в бассейне 25 м. После того как первый участник соревнований проплыл часть дорожки, ему осталось проплыть 10 м. Сколько метров уже проплыл участник соревнований?'
#sentence = 'Используя данные круговой диаграммы, реши задачу. На диаграмме представлены данные о турпоходе группы. Сколько километров прошла группа в четвёртый день? Введи в поле ответа число без единиц измерения. При необходимости ввести десятичную дробь, разделяй её целую и дробную части запятой, без пробелов.'
#sentence = 'Во дворе кот Геннадий охотился на напыщенного толстого голубя. (1) крадущегося кота была равна 1 м/с. Но голубь оказался не так прост и скоро, через (2), заметил приближающегося Геннадия. Птица взлетела на крышу гаража высотой в (3), чтобы отвязаться от кота. Но тот быстро преодолел (4) до этого строения в 5 метров, прыгнул наверх и почти ухватил пернатого за хвост! Голубь, конечно, не стерпел такой наглости и взлетел со скоростью (5). Обиженный Геннадий посмотрел некоторое (6) на улетающую птицу и спрыгнул вниз. Какова длина траектории (в метрах) кота Геннадия за описанную утреннюю прогулку?'
#sentence = 'У Миши было 3 три мячика. Два из них он отдал Даше. Сколько мячиков осталось у Миши?'
category_from = 2
category_to = 0

In [ ]:
words_in_sentence = word_tokenize(sentence)
for ind, word in enumerate(words_in_sentence):
  word = word.lower()
  if not word in navec:
    continue
  pred = word_model.predict(np.array([navec[word]]))[0]
  ind_max = np.argmax(pred)
  if pred[ind_max].item() > 0.9 and ind_max == category_from:
    print(word)

круговой
поле


In [130]:
print(sentence)
sentence_with_masks, masks = MaskCreator().mask(sentence)
#print(sentence_with_masks, masks)
new_masks = ['' for i in range(len(masks))]
words_in_sentence = word_tokenize(sentence_with_masks)
get_first_mask = False
for word in words_in_sentence:
  if word[:4] == 'mask' and masks[int(word[4:])] in nouns_actors[category_from]:
    first_mask = navec[masks[int(word[4:])]]
    new_word = nouns_actors[category_to][random.randint(0, 19)]
    #new_word = masks[int(word[4:])]
    new_first_mask = navec[new_word]
    #new_first_mask = first_mask
    new_masks[int(word[4:])] = new_word
    get_first_mask = True
for ind, word in enumerate(words_in_sentence):
  if word[:4] == 'mask':
    if not get_first_mask:
      first_mask = navec[masks[int(word[4:])]]
      new_word = nouns[category_to][random.randint(0, 19)]
      #new_word = masks[int(word[4:])]
      new_first_mask = navec[new_word]
      #new_first_mask = first_mask
      new_masks[int(word[4:])] = new_word
      get_first_mask = True
    elif new_masks[int(word[4:])] == '':
      min_val = 1
      min_new_word = ''
      ideal_dist = first_mask - navec[masks[int(word[4:])]]
      for new_word in nouns[category_to]:
        if not new_word in navec or new_word in new_masks:
          continue
        now_dist = cos_dist.cosine(navec[new_word], new_first_mask - ideal_dist)
        if now_dist < min_val:
          min_val = now_dist
          min_new_word = new_word
      new_masks[int(word[4:])] = min_new_word
    words_in_sentence[ind] = new_masks[int(word[4:])]

sentence = TreebankWordDetokenizer().detokenize(words_in_sentence)
sentence = sentence.replace(' .', '.')

sentence

   1.4. Во время летних соревнований по плаванию ребята посетили бассейн. Длина плавательной дорожки в бассейне 25 м. После того как первый участник соревнований проплыл часть дорожки, ему осталось проплыть 10 м. Сколько метров уже проплыл участник соревнований?


'1. 4. во время летних кашалот по парусник животное посетили аквариум. длина плавательной речка в аквариум 25 м. после того как первый дельфин кашалот проплыл часть речка, ему осталось проплыть 10 м. сколько метр уже проплыл дельфин кашалот?'

In [ ]:
words_in_sentence = word_tokenize(sentence)
to_word = {}
count_masks = 0
for ind, word in enumerate(words_in_sentence):
  if morph.parse(word)[0].tag.POS == 'INFN' or morph.parse(word)[0].tag.POS == 'VERB':
    words_in_sentence[ind] = '[MASK]'
    count_masks += 1
    continue
  '''
  normal_word = morph.parse(word)[0].normal_form
  case_word = morph.parse(word)[0].tag.case
  number_word = morph.parse(word)[0].tag.number
  if normal_word in nouns_actors[category_from]:
    if to_word.get(normal_word) == None:
      to_word[normal_word] = nouns_actors[category_to][random.randint(0, 19)]
    # Заменяю либо на существительное из другой категории, либо на маску (тестировал разные варианты)
    #words_in_sentence[ind] = morph.parse(to_word[normal_word])[0].inflect({number_word, case_word})[0]
    words_in_sentence[ind] = '[MASK]'
    count_masks += 1
  if normal_word in nouns[category_from]:
    if to_word.get(normal_word) == None:
      to_word[normal_word] = nouns[category_to][random.randint(0, 19)]
    #words_in_sentence[ind] = morph.parse(to_word[normal_word])[0].inflect({number_word, case_word})[0]
    words_in_sentence[ind] = '[MASK]'
    count_masks += 1'''

In [ ]:
sentence = TreebankWordDetokenizer().detokenize(words_in_sentence)
sentence = sentence.replace(' .', '.')

sentence

'1. 4. Во время летних соревнование по океан папа [MASK] пруд. длина плавательной полоса в пруд 25 м. После того как первый художник соревнование [MASK] часть полоса, ему [MASK] [MASK] 10 м. Сколько метр уже [MASK] художник соревнование?'

In [ ]:
tokenized = tokenizer(sentence, return_tensors="pt")
with torch.no_grad():
  outputs = model(**tokenized)
masked_index = np.where(torch.flatten(tokenized['input_ids']).numpy() == 103)
for i in masked_index[0]:
  predicted_index = torch.argmax(outputs[0][0][i]).item()
  predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
  ind = sentence.split().index('[MASK]')
  sentence = sentence.split()
  sentence[ind] = predicted_token
  sentence = ' '.join(sentence)

In [ ]:
sentence

'1. 4. Во время летних соревнование по океан папа по пруд. длина плавательной полоса в пруд 25 м. После того как первый художник соревнование . часть полоса, ему длина ##о 10 м. Сколько метр уже первый художник соревнование?'